In [54]:
import numpy as np 
import pandas as pd
import seaborn as sns

# Description of the dataset

**Source of dataset:** https://www.kaggle.com/datasets/zynicide/wine-reviews

**Number of observations**

**Number of variables per observation**



In [55]:
df = pd.read_csv('winemag-data-130k-v2.csv')
df.sample(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
69183,69183,US,"Good for drinking now, it's bright in acidity,...",NaN,87,24.0,California,Sonoma Valley,Sonoma,NaN,NaN,Lake Sonoma 2012 Sauvignon Blanc (Sonoma Valley),Sauvignon Blanc,Lake Sonoma
118031,118031,Macedonia,This Merlot-Cabernet-Vranec blend offers an ap...,Alexandar,86,16.0,Tikves,NaN,NaN,Susan Kostrzewa,@suskostrzewa,Bovin 2006 Alexandar Red (Tikves),Red Blend,Bovin
11164,11164,US,"Tastes more like a Zin than a Cab, with rustic...",Rockpile Vineyard,84,38.0,California,Sonoma County,Sonoma,NaN,NaN,Paradise Ridge 2003 Rockpile Vineyard Cabernet...,Cabernet Sauvignon,Paradise Ridge
96532,96532,US,"Lots of decadent, butterscotch-like oakiness m...",NaN,85,17.0,California,California,California Other,NaN,NaN,Butternut 2011 Chardonnay (California),Chardonnay,Butternut
71119,71119,Italy,"A blend of three native grapes (Trebbiano, Pec...",Villa Prandone Aurato,86,NaN,Central Italy,Falerio dei Colli Ascolani,NaN,NaN,NaN,Il Conte 2005 Villa Prandone Aurato White (Fal...,White Blend,Il Conte


**Meaning and type of the different variables**

In [56]:
df.dtypes

Unnamed: 0                 int64
country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

# Data Cleaning

The column "Unnamed: 0" is useless because count the number of the row 

Also 'taster_name' and 'taster_twitter_handle' are useless for us

In [57]:
df.drop(columns=['Unnamed: 0', 'taster_name', 'taster_twitter_handle'], inplace=True)

In [58]:
for column in df.columns:
    nan_count = df[column].isna().sum()
    unique_count = df[column].nunique()
    print(f"Column '{column}':")
    print(f"  Missing values: {nan_count}")
    print(f"  Unique values: {unique_count}")
    print()

Column 'country':
  Missing values: 63
  Unique values: 43

Column 'description':
  Missing values: 0
  Unique values: 119955

Column 'designation':
  Missing values: 37465
  Unique values: 37979

Column 'points':
  Missing values: 0
  Unique values: 21

Column 'price':
  Missing values: 8996
  Unique values: 390

Column 'province':
  Missing values: 63
  Unique values: 425

Column 'region_1':
  Missing values: 21247
  Unique values: 1229

Column 'region_2':
  Missing values: 79460
  Unique values: 17

Column 'title':
  Missing values: 0
  Unique values: 118840

Column 'variety':
  Missing values: 1
  Unique values: 707

Column 'winery':
  Missing values: 0
  Unique values: 16757



### Country column analysis

In [59]:
# Filter the DataFrame to show only rows where the specified column has NaN values
rows_with_nan = df[df['country'].isna()]
rows_with_nan

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
913,NaN,"Amber in color, this wine has aromas of peach ...",Asureti Valley,87,30.0,NaN,NaN,NaN,Gotsa Family Wines 2014 Asureti Valley Chinuri,Chinuri,Gotsa Family Wines
3131,NaN,"Soft, fruity and juicy, this is a pleasant, si...",Partager,83,NaN,NaN,NaN,NaN,Barton & Guestier NV Partager Red,Red Blend,Barton & Guestier
4243,NaN,"Violet-red in color, this semisweet wine has a...",Red Naturally Semi-Sweet,88,18.0,NaN,NaN,NaN,Kakhetia Traditional Winemaking 2012 Red Natur...,Ojaleshi,Kakhetia Traditional Winemaking
9509,NaN,This mouthwatering blend starts with a nose of...,Theopetra Malagouzia-Assyrtiko,92,28.0,NaN,NaN,NaN,Tsililis 2015 Theopetra Malagouzia-Assyrtiko W...,White Blend,Tsililis
9750,NaN,This orange-style wine has a cloudy yellow-gol...,Orange Nikolaevo Vineyard,89,28.0,NaN,NaN,NaN,Ross-idi 2015 Orange Nikolaevo Vineyard Chardo...,Chardonnay,Ross-idi
...,...,...,...,...,...,...,...,...,...,...,...
124176,NaN,This Swiss red blend is composed of four varie...,Les Romaines,90,30.0,NaN,NaN,NaN,Les Frères Dutruy 2014 Les Romaines Red,Red Blend,Les Frères Dutruy
129407,NaN,Dry spicy aromas of dusty plum and tomato add ...,Reserve,89,22.0,NaN,NaN,NaN,El Capricho 2015 Reserve Cabernet Sauvignon,Cabernet Sauvignon,El Capricho
129408,NaN,El Capricho is one of Uruguay's more consisten...,Reserve,89,22.0,NaN,NaN,NaN,El Capricho 2015 Reserve Tempranillo,Tempranillo,El Capricho
129590,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,NaN,NaN,Büyülübağ 2012 Shah Red,Red Blend,Büyülübağ


We have no knowledge of the country or the province or the region, so we have no idea of the origin of this wine.

### Drop duplicates

In total there are 129971 wine and is strange that the column "description" has 0 Missing values and 119955 Unique values

In [60]:
total_entries = df['description'].shape[0]
unique_entries = df['description'].nunique()

# Calculate the number of duplicate entries
duplicate_entries = total_entries - unique_entries

print(f"Number of duplicate entries in '{'description'}': {duplicate_entries}")

Number of duplicate entries in 'description': 10016


In [61]:
duplicated = df.duplicated().sum()
print(f"Total duplicate: {duplicated}")
df.drop_duplicates(inplace=True)

Total duplicate: 9983


In [62]:
total_entries = df['description'].shape[0]
unique_entries = df['description'].nunique()

# Calculate the number of duplicate entries
duplicate_entries = total_entries - unique_entries

print(f"Number of duplicate entries in '{'description'}': {duplicate_entries}")

Number of duplicate entries in 'description': 33


### Handle NaN

We want to replace NaN value with 0 in the Price column

In [63]:
nan_count = df['price'].isna().sum()
print(f"total NaN in price: {nan_count}")
df['price'] = df['price'].fillna(0).astype('float64')
nan_count = df['price'].isna().sum()
print(f"total NaN in price: {nan_count}")

8395
0
